In [11]:
from scripts.plotting import read_and_plot_scenarios

import numpy as np
import pandas as pd

Inputs:

In [356]:
#################### INPUT ####################
renewable = "solar"

demand = 100 # GWa
primary_resource = 40 # GWa
secondary_resource = 90 # GWa
LDES = 0 # GWa 
SDES = 0 # GWa

ACT = {"ldes":LDES,
       "sdes":SDES}
###############################################

Read gamma coefficients:

In [357]:
filenames = {"wind":"wind_curtailment_delta_df1_dx1",
             "solar":"solar_curtailment_delta_df2_dx2"}
gamma_ij = pd.read_csv("MESSAGEix_GLOBIOM/" + filenames[renewable] + ".csv",index_col=0)
gamma_ij["ind"] = gamma_ij.index
gamma_ij["ws_ind"] = gamma_ij.ind.str.split("_",expand=True)[2]
if renewable == "wind":
    gamma_ij["solar"] = gamma_ij["ws_ind"].str.split("s",expand=True)[1].astype(int)
    gamma_ij["w_ind"] = gamma_ij["ws_ind"].str.split("s",expand=True)[0]
    gamma_ij["wind"] = gamma_ij["w_ind"].str.split("w",expand=True)[1].astype(int)
    gamma_ij.drop(columns=["ind","ws_ind","w_ind"],inplace=True)
elif renewable == "solar":
    gamma_ij["wind"] = gamma_ij["ws_ind"].str.split("w",expand=True)[1].astype(int)
    gamma_ij["s_ind"] = gamma_ij["ws_ind"].str.split("w",expand=True)[0]
    gamma_ij["solar"] = gamma_ij["s_ind"].str.split("s",expand=True)[1].astype(int)
    gamma_ij.drop(columns=["ind","ws_ind","s_ind"],inplace=True)

gamma_ij.set_index(["solar","wind"],inplace=True)

Read beta coefficients:

In [334]:
tech = "ldes"
act_tech = ACT[tech]
beta_tech_renewable = pd.read_csv("MESSAGEix_GLOBIOM/beta_" + tech + "_" + renewable + ".csv",index_col=0)

beta_tech_renewable.columns = [0,0.1,0.3,0.4,0.5,0.6,0.7]
beta_tech_renewable.index = [0,0.1,0.3,0.4,0.5,0.6,0.7]

beta_tech_renewable_explanatory = beta_tech_renewable.copy()
beta_tech_renewable_explanatory.columns = ["0",">=10%",">=30%",">=40%",">=50%",">=60%",">=70%"]
beta_tech_renewable_explanatory.index = ["0",">=10%",">=30%",">=40%",">=50%",">=60%",">=70%"]

Equations for the base curtailment:

In [359]:
phi_i = [0, 10, 30, 40, 50, 60, 70, 90]
phi_j = [10, 30, 40, 50, 60, 70]

# base term
curtailment_base = {}
for i in range(len(phi_i)-1):
    curtailment_base[0,i] = gamma_ij.loc[0,i]*(primary_resource - phi_i[i]) if primary_resource > phi_i[i] else 0
    
curtailment_base_df = pd.DataFrame.from_dict(curtailment_base).T.sort_index()
base_term_sum = curtailment_base_df.sum() 

# secondary resource term
curtailment_ij = {}
for i in range(len(phi_i)-1):
    for j in range(len(phi_j)):
        if (j+1,i) in gamma_ij.index:
            curtailment = gamma_ij.loc[j+1,i]*(secondary_resource - phi_j[j]) if secondary_resource > phi_j[j] else 0
            curtailment_ij[j,i] = curtailment if primary_resource >= phi_i[i+1] else 0
            
curtailment_ij_df = pd.DataFrame.from_dict(curtailment_ij).T.sort_index()
secondary_term_sum = curtailment_ij_df.sum()

Equations for the technology term:

In [360]:
# tech term 
curtailment_tech_ij = {}
bins_i = beta_tech_renewable.columns
bins_j = beta_tech_renewable.index

for i in range(len(bins_i)):
    for j in range(len(bins_j)):

        if j < len(bins_j)-1 and i < len(bins_i)-1:
            condition_i = primary_resource >= bins_i[i]*demand and primary_resource < bins_i[i+1]*demand
            condition_j = secondary_resource >= bins_j[j]*demand and secondary_resource < bins_j[j+1]*demand
        
        elif j == len(bins_j)-1 and i < len(bins_i)-1: # boundary of secondary resource bins
            condition_i = primary_resource >= bins_i[i]*demand and primary_resource < bins_i[i+1]*demand
            condition_j = secondary_resource >= bins_j[j]*demand

        elif i == len(bins_i)-1 and j < len(bins_j)-1: # boundary of primary resource bins
            condition_i = primary_resource >= bins_i[i]*demand
            condition_j = secondary_resource >= bins_j[j]*demand and secondary_resource < bins_j[j+1]*demand
        
        elif i == len(bins_i)-1 and j == len(bins_j)-1: # boundary of both primary and secondary resource bins
            condition_i = primary_resource >= bins_i[i]*demand
            condition_j = secondary_resource >= bins_j[j]*demand

        curtailment_tech_ij[j,i] = beta_tech_renewable.loc[bins_j[j],bins_i[i]]*act_tech if condition_i and condition_j else 0

tech_term_series = pd.Series(curtailment_tech_ij).sort_index()
tech_term_sum = tech_term_series.sum()

Calculate total curtailment:

In [361]:
total_curtailment = base_term_sum + secondary_term_sum + tech_term_sum # percentage of demand

In [363]:
base_term_sum

0    0.070398
dtype: float64

In [364]:
secondary_term_sum

0    60.036766
dtype: float64

In [362]:
total_curtailment

0    60.107164
dtype: float64